In [10]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# We need to import key libraries that we're going to use.  
# For now this is just numpy, which is our linear algebra library
import numpy as np
import pandas as pd
# to make this notebook's output stable across runs, we are going to see the random seed
np.random.seed(42)

# To plot pretty figures
import matplotlib.pyplot as plt
# %matplotlib inline 
# This command figures show up in the notebook.  It's a "magic" command...
# Typically, this now happens by default so it is often an unnecessary command, but is good for standardization.
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15

# Read HDF5
import h5py

from decimal import Decimal

from numpy import zeros

from sklearn import preprocessing
from sklearn import pipeline # 


from scipy import stats


from mpl_toolkits.mplot3d import Axes3D # this part works well in py3
from matplotlib import cm

# from mpl_toolkits import mplot3d
# import pylab as p

from IPython.display import Image

from sklearn.metrics import mean_squared_error,r2_score

from sklearn import model_selection

import time

from sklearn import linear_model

import pickle

In [2]:
PROJECT_ROOT_DIR = "."
Folder_DNS_Results = "results_N_3_V_17"
DirectoryTo_DNS_Results=(PROJECT_ROOT_DIR+'/'+Folder_DNS_Results)
print("DirectoryTo_DNS_Results:{}".format(DirectoryTo_DNS_Results))


DirectoryTo_DNS_Results:./results_N_3_V_17


In [3]:
PROJECT_SAVE_DIR = "Reza_ML_Saved_PHProject"
# makes the directory if it doesn't exist.
import os
if not (os.path.isdir(PROJECT_ROOT_DIR+'/'+PROJECT_SAVE_DIR)):
    print('Figure directory didn\'t exist, creating now.')
    os.mkdir(PROJECT_ROOT_DIR+'/'+PROJECT_SAVE_DIR)
else:
    print('Figure directory exists.') 

Figure directory exists.


In [4]:
#a simple defined helper function.
def savepdf(fig,name):
    fig.savefig(PROJECT_ROOT_DIR+'/'+PROJECT_SAVE_DIR+'/'+name+'.pdf',transparent=True)

In [5]:
def savemodel_pkl(model,pkl_filename):
    # Save to file in the current working directory
    # pkl_filename = "pickle_model.pkl"  
    with open(PROJECT_ROOT_DIR+'/'+PROJECT_SAVE_DIR+'/'+pkl_filename+'.pkl', 'wb') as file:  
        pickle.dump(model, file)

In [6]:
def loadmodel_pkl(pkl_filename):
    # Save to file in the current working directory
    # pkl_filename = "pickle_model.pkl"  
    with open(PROJECT_ROOT_DIR+'/'+PROJECT_SAVE_DIR+'/'+pkl_filename+'.pkl', 'rb') as file:  
        b=pickle.load(file)
        return b

In [7]:

N=128; # number of grid points in the DNS setup
DNSdata_time_Step=8000
A11=np.zeros((N,N,N))
A12=np.zeros((N,N,N))
A13=np.zeros((N,N,N))
A21=np.zeros((N,N,N))
A22=np.zeros((N,N,N))
A23=np.zeros((N,N,N))
A31=np.zeros((N,N,N))
A32=np.zeros((N,N,N))
A33=np.zeros((N,N,N))

A2_11=np.zeros((N,N,N))
A2_12=np.zeros((N,N,N))
A2_13=np.zeros((N,N,N))
A2_21=np.zeros((N,N,N))
A2_22=np.zeros((N,N,N))
A2_23=np.zeros((N,N,N))
A2_31=np.zeros((N,N,N))
A2_32=np.zeros((N,N,N))
A2_33=np.zeros((N,N,N))

Trace_A2=np.zeros((N,N,N))
Trace_A3=np.zeros((N,N,N))

H11=np.zeros((N,N,N))
H12=np.zeros((N,N,N))
H13=np.zeros((N,N,N))
H22=np.zeros((N,N,N))
H23=np.zeros((N,N,N))
H33=np.zeros((N,N,N))


i_coordinate=np.zeros((N,N,N)) # depth direction
j_coordinate=np.zeros((N,N,N)) # column direction
k_coordinate=np.zeros((N,N,N)) # row direction

# print(type(A11))
# print(A11.shape)
# print(A11[1,1,1])

all_data = [] 
df = pd.DataFrame([])

file_TimeStep=('%06.0f' % Decimal(DNSdata_time_Step))

for b in range(0,16): # (0,16)
    file_Rank=('%06.0f' % Decimal(b))
    filename=('VelG_R.'+file_TimeStep+'.h5.'+file_Rank)
    filename_PH=('PressureH.'+file_TimeStep+'.h5.'+file_Rank)
    
    print(filename)
    f = h5py.File(DirectoryTo_DNS_Results+'/'+filename, 'r')
    g=h5py.File(DirectoryTo_DNS_Results+'/'+filename_PH, 'r')
    
    i_cell_ID= f['i_cell_ID']
    j_cell_ID= f['j_cell_ID']
    k_cell_ID= f['k_cell_ID']
    
    i_cell_ID_max=np.amax(i_cell_ID )
    i_cell_ID_min=np.amin(i_cell_ID )
    j_cell_ID_max=np.amax(j_cell_ID )
    j_cell_ID_min=np.amin(j_cell_ID )
    k_cell_ID_max=np.amax(k_cell_ID )
    k_cell_ID_min=np.amin(k_cell_ID )
    
#     print("f['dUdxG_R'].shape: {}".format(f['dUdxG_R'].shape)) # to see how the data is stored here 
#     print("f['dUdxG_R'].size: {}".format(f['dUdxG_R'].size)) # to see how the data is stored here 
    
    
#     print("i_cell_ID_max:{}".format(i_cell_ID_max))
#     print("i_cell_ID_min:{}".format(i_cell_ID_min))
#     print("j_cell_ID_max:{}".format(j_cell_ID_max))
#     print("j_cell_ID_min:{}".format(j_cell_ID_min))
#     print("k_cell_ID_max:{}".format(k_cell_ID_max))
#     print("k_cell_ID_min:{}".format(k_cell_ID_min))
    
    i_coordinate[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=i_cell_ID
    j_coordinate[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=j_cell_ID
    k_coordinate[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=k_cell_ID
    
    # all_data.append(f['dUdyG_R'])
    #df = df.append(pd.DataFrame({'A12': f['dUdyG_R']}))
    # A11[k_cell_ID_min:k_cell_ID_max][j_cell_ID_min:j_cell_ID_max][i_cell_ID_min:i_cell_ID_max]=f['dUdxG_R']
    A11[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['dUdxG_R'][:]
    A12[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['dUdyG_R'][:]
    A13[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['dUdzG_R'][:]
    A21[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['dVdxG_R'][:]
    A22[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['dVdyG_R'][:]
    A23[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['dVdzG_R'][:]
    A31[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['dWdxG_R'][:]
    A32[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['dWdyG_R'][:]
    A33[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['dWdzG_R'][:]
    
    A2_11[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['A2_11_R'][:]
    A2_12[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['A2_12_R'][:]
    A2_13[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['A2_13_R'][:]
    A2_21[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['A2_21_R'][:]
    A2_22[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['A2_22_R'][:]
    A2_23[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['A2_23_R'][:]
    A2_31[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['A2_31_R'][:]
    A2_32[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['A2_32_R'][:]
    A2_33[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['A2_33_R'][:]
    
    
    Trace_A2[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['trace_A2'][:]
    Trace_A3[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=f['trace_A3'][:]
    
    H11[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=g['H11'][:]
    H12[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=g['H12'][:]
    H13[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=g['H13'][:]
    H22[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=g['H22'][:]
    H23[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=g['H23'][:]
    H33[k_cell_ID_min-1:k_cell_ID_max, j_cell_ID_min-1:j_cell_ID_max, i_cell_ID_min-1:i_cell_ID_max]=g['H33'][:]
    
    
    #A12(k_cell_ID_min:k_cell_ID_max,j_cell_ID_min:j_cell_ID_max,i_cell_ID_min:i_cell_ID_max)=f['dUdyG_R']
     

VelG_R.008000.h5.000000
VelG_R.008000.h5.000001
VelG_R.008000.h5.000002
VelG_R.008000.h5.000003
VelG_R.008000.h5.000004
VelG_R.008000.h5.000005
VelG_R.008000.h5.000006
VelG_R.008000.h5.000007
VelG_R.008000.h5.000008
VelG_R.008000.h5.000009
VelG_R.008000.h5.000010
VelG_R.008000.h5.000011
VelG_R.008000.h5.000012
VelG_R.008000.h5.000013
VelG_R.008000.h5.000014
VelG_R.008000.h5.000015


In [8]:
A = [A11, A12, A13, A21, A22, A23, A31, A32, A33]
A2 = [A2_11, A2_12, A2_13, A2_21, A2_22, A2_23, A2_31, A2_32, A2_33]
H=[H11,H12,H13,H22,H23,H33]
coordinate_RCD=[k_coordinate, j_coordinate, i_coordinate] # Row Column Depth
Q_R_data= np.zeros((N**3, 2)) # 
Atot = np.zeros((N**3, 9))
A2_tot = np.zeros((N**3, 9))
Htot = np.zeros((N**3, 6))
coordinate = np.zeros((N**3, 3))
for i in range(9):
    Atot[:,i] = np.reshape(A[i], N**3)
    A2_tot[:,i] = np.reshape(A2[i], N**3)
    if i<6:
        Htot[:,i] = np.reshape(H[i], N**3)
        if i<3:
            coordinate[:,i] = np.reshape(coordinate_RCD[i], N**3)
            if i<1:
                Q_R_data[:,i]=np.reshape(-0.5*Trace_A2, N**3)
                Q_R_data[:,i+1]=np.reshape(-0.5*Trace_A3, N**3)

In [9]:
h5f = h5py.File(PROJECT_ROOT_DIR+'/'+PROJECT_SAVE_DIR+'/'+'Q_R_A_A2_H_data_Ts_'+str(DNSdata_time_Step)+'.h5', "w")
h5f.create_dataset('dataset_Q_R_data', data=Q_R_data)
h5f.create_dataset('dataset_Atot', data=Atot)
h5f.create_dataset('dataset_A2_tot', data=A2_tot)
h5f.create_dataset('dataset_Htot', data=Htot)
h5f.close()